<a href="https://colab.research.google.com/github/Deven10103/MachineLearning.explore/blob/main/LLMs/TensorRT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install opencv-python
!pip install numpy
!pip install torch-tensorrt==<2.3.0>
!pip install pandas
!pip install matplotlib

/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `pip install torch-tensorrt==<2.3.0>'


In [1]:

import torch
# import torch_tensorrt

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-6-25 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


cuda


In [2]:
!wget https://data.vision.ee.ethz.ch/csakarid/shared/SFSU_synthetic/Downloads/Foggy_Driving.zip
!unzip Foggy_Driving.zip

--2025-06-25 10:08:59--  https://data.vision.ee.ethz.ch/csakarid/shared/SFSU_synthetic/Downloads/Foggy_Driving.zip
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 100649749 (96M) [application/zip]
Saving to: ‘Foggy_Driving.zip’

Foggy_Driving.zip   100%[===================>]  95.99M  22.0MB/s    in 5.6s    

2025-06-25 10:09:06 (17.2 MB/s) - ‘Foggy_Driving.zip’ saved [100649749/100649749]

Archive:  Foggy_Driving.zip
   creating: Foggy_Driving/
   creating: Foggy_Driving/gtCoarse/
   creating: Foggy_Driving/gtCoarse/test_extra/
   creating: Foggy_Driving/gtCoarse/test_extra/web/
  inflating: Foggy_Driving/gtCoarse/test_extra/web/web_20080313_Foggy_gtCoarse_labelIds.png  
  inflating: Foggy_Driving/gtCoarse/test_extra/web/web_5378529177_51ff165684_gtCoarse_labelIds.png  
  inflating: Fo

In [3]:
import os
import cv2
# import numpy as np

def preProcess(img_paths):
  images=[]
  for img_path in img_paths:
    img = cv2.imread(img_path)

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(640,640))

    img = img.transpose(2,0,1)
    img = torch.from_numpy(img).to(device)
    img = img.float()

    images.append(img)
  return images


In [13]:

image_dir = 'Foggy_Driving/leftImg8bit/test/public'
image_paths = [os.path.join(image_dir, img) for img in os.listdir(image_dir)]

IMAGE_NUMS = 128
# print(image_paths)
images = preProcess(image_paths[:IMAGE_NUMS])
imagesONNX = image_paths[:IMAGE_NUMS]
print(imagesONNX)

['Foggy_Driving/leftImg8bit/test/public/public_20161213_082948_leftImg8bit.png', 'Foggy_Driving/leftImg8bit/test/public/public_20161213_082952_leftImg8bit.png', 'Foggy_Driving/leftImg8bit/test/public/public_20161213_081817_leftImg8bit.png', 'Foggy_Driving/leftImg8bit/test/public/public_20161213_082937_leftImg8bit.png', 'Foggy_Driving/leftImg8bit/test/public/public_20161213_082426_leftImg8bit.png', 'Foggy_Driving/leftImg8bit/test/public/public_20161213_081724_leftImg8bit.png', 'Foggy_Driving/leftImg8bit/test/public/public_20161213_082117_leftImg8bit.png', 'Foggy_Driving/leftImg8bit/test/public/public_20161213_081721_leftImg8bit.png', 'Foggy_Driving/leftImg8bit/test/public/public_20161213_082012_leftImg8bit.png', 'Foggy_Driving/leftImg8bit/test/public/public_20161213_082109_leftImg8bit.png', 'Foggy_Driving/leftImg8bit/test/public/public_20161213_081801_leftImg8bit.png', 'Foggy_Driving/leftImg8bit/test/public/public_20161213_083206_leftImg8bit.png', 'Foggy_Driving/leftImg8bit/test/public/

In [7]:
import time

def runInference(model,images):
  times=[]
  results=[]
  for image in images:
    image = image.unsqueeze(0)
    image_gpu=image.to(device)
    # print(device)
    start=time.time()

    # input_array = image_gpu.cpu().numpy().astype(np.float32)
    # result = model.run(None, {"input_name": input_array})

    with torch.no_grad():
      result=model(image_gpu)

    results.append(result)
    times.append(time.time()-start)
  return results, times

In [8]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

trials = 5
avgs=[]

for i in range(trials):
  results, times = runInference(model,images)
  # print(times)
  avg=sum(times) / len(times)
  print("Average inference time (s):", avg)

  avgs.append(avg)

mat = sum(avgs)/trials
print("Mean average inference time (s):", mat)

Average inference time (s): 0.012069419578269677
Average inference time (s): 0.0070068129786738645
Average inference time (s): 0.006015680454395435
Average inference time (s): 0.00667075757627134
Average inference time (s): 0.006653423662538882
Mean average inference time (s): 0.007683218850029841


# **Yolo to ONNX :**

In [9]:
# 1. Install dependencies
!pip install onnx onnxruntime-gpu

# 2. Import modules
import torch
import torchvision.models as models
import onnx
import onnxruntime as ort
import numpy as np

# 3. Get a sample model (ResNet18) and convert to ONNX
# model = models.resnet18(pretrained=True).eval()
dummy_input = torch.randn(1, 3, 224, 224)
torch.onnx.export(model, dummy_input, "ONNXModel.onnx",
                  input_names=["input"], output_names=["output"],
                  opset_version=12)

# 4. Load ONNX model with GPU execution provider (TensorRT if available, else CUDA)
providers = ['CUDAExecutionProvider']
session = ort.InferenceSession("ONNXModel.onnx", providers=providers)

# 5. Run inference
input_array = dummy_input.numpy()
outputs = session.run(None, {"input": input_array})
print("Output shape:", outputs[0].shape)




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.2/283.2 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.3 MB/s eta 0:00:00


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:700: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  y = self.model(im, augment=augment, visualize=visualize) if augment or visualize else self.model(im)
/root/.cache/torch/hub/ultralytics_yolov5_master/models/yolo.py:101: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.dynamic or self.grid[i].shape[2:4] != x[i].shape[2:4]:


Output shape: (1, 3087, 85)


In [18]:
import time

def runInferenceONNX(model,images):
  times=[]
  results=[]
  for image in images:
    # image = image.unsqueeze(0)
    # image_gpu=image.to(device)
    # print(device)
    start=time.time()

    # input_array = image_gpu.cpu().numpy().astype(np.float32)
    result = model.predict(image,save=True)

    # with torch.no_grad():
    #   result=model(input_array)

    results.append(result)
    times.append(time.time()-start)
  return results, times

In [19]:
!pip install ultralytics
from ultralytics import YOLO
import onnxruntime as ort

Newmodel = YOLO("ONNXModel.onnx")
session = ort.InferenceSession("ONNXModel.onnx", providers=providers)

trials = 5
avgs=[]

for i in range(trials):
  results, times = runInferenceONNX(Newmodel,imagesONNX)
  # print(times)
  avg=sum(times) / len(times)
  print("Average inference time (s):", avg)

  avgs.append(avg)

mat = sum(avgs)/trials
print("Mean average inference time (s):", mat)

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading ONNXModel.onnx for ONNX Runtime inference...
Using ONNX Runtime CUDAExecutionProvider
WARNING ⚠️ Metadata not found for 'model=ONNXModel.onnx'



RuntimeError: Error when binding input: Failed to find input name in the mapping: images

# **Yolo in TensorRT:**

In [ ]:
import torch_tensorrt

model.eval()  # Set the model to evaluation mode
input_data = torch.randn(1, 3, 640, 640)


trt_model = torch_tensorrt.compile(model, inputs=[input_data],
                                   ir='ts',
                                   enabled_precisions={torch.float32},
                                   truncate_long_and_double=True)

/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:700: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  y = self.model(im, augment=augment, visualize=visualize) if augment or visualize else self.model(im)
/root/.cache/torch/hub/ultralytics_yolov5_master/models/yolo.py:101: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.dynamic or self.grid[i].shape[2:4] != x[i].shape[2:4]:


In [ ]:
print("TensorRT optimized")

avgs = []
for i in range(NUM_TRIALS):
    trt_results, trt_time = run_inference(trt_model, images)
    avg = sum(trt_time)/len(images)
    print("Average inference time (s):", avg)
    avgs.append(avg)

trt_mat = sum(avgs)/NUM_TRIALS
print("Mean average inference time (s):", trt_mat)

Output shape: (1, 3087, 85)


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

trials = 5
avgs=[]

for i in range(trials):
  results, times = runInferenceONNX(,images)
  # print(times)
  avg=sum(times) / len(times)
  print("Average inference time (s):", avg)

  avgs.append(avg)

mat = sum(avgs)/trials
print("Mean average inference time (s):", mat)

TypeError: 'NoneType' object is not callable

In [ ]:
import torch_tensorrt

model.eval()
input_data= torch.randn(1,3,640,640)

trtModel=torch_tensorrt.compile(model, inputs=[input_data],
                                   ir='ts',
                                   enabled_precisions={torch.float32},
                                   truncate_long_and_double=True)